# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

rating_map = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

def scrape_books(min_rating, max_price):
    base_url = "https://books.toscrape.com/catalogue/"
    page_url = "https://books.toscrape.com/catalogue/page-1.html"
    
    books_data = []

    while True:
        print(f"Scrapeando: {page_url}")
        response = requests.get(page_url)
        response.encoding = 'utf-8'  # ← FIX IMPORTANTE
        soup = BeautifulSoup(response.text, "html.parser")

        books = soup.select("article.product_pod")

        for book in books:
            rating_class = book.find("p", class_="star-rating")["class"][1]
            rating = rating_map[rating_class]

            if rating < min_rating:
                continue

            book_url = base_url + book.find("a")["href"].replace("../", "")

            book_resp = requests.get(book_url)
            book_resp.encoding = 'utf-8'  # ← FIX IMPORTANTE
            book_soup = BeautifulSoup(book_resp.text, "html.parser")

            title = book_soup.find("h1").text.strip()

            table = book_soup.find("table", class_="table-striped")
            rows = {row.th.text: row.td.text for row in table.find_all("tr")}
            upc = rows.get("UPC")

            price_text = rows.get("Price (incl. tax)").replace("£", "").strip()
            price = float(price_text)

            if price > max_price:
                continue

            availability = rows.get("Availability")

            genre = book_soup.select("ul.breadcrumb li a")[2].text.strip()

            desc_tag = book_soup.find("div", id="product_description")
            if desc_tag:
                description = desc_tag.find_next_sibling("p").text.strip()
            else:
                description = "No disponible"

            books_data.append({
                "UPC": upc,
                "Title": title,
                "Price (£)": price,
                "Rating": rating,
                "Genre": genre,
                "Availability": availability,
                "Description": description
            })

        next_button = soup.find("li", class_="next")
        if not next_button:
            break

        next_page = next_button.find("a")["href"]
        page_url = "https://books.toscrape.com/catalogue/" + next_page

    return pd.DataFrame(books_data)


In [8]:

df = scrape_books(min_rating=4, max_price=20)
df



Scrapeando: https://books.toscrape.com/catalogue/page-1.html
Scrapeando: https://books.toscrape.com/catalogue/page-2.html
Scrapeando: https://books.toscrape.com/catalogue/page-3.html
Scrapeando: https://books.toscrape.com/catalogue/page-4.html
Scrapeando: https://books.toscrape.com/catalogue/page-5.html
Scrapeando: https://books.toscrape.com/catalogue/page-6.html
Scrapeando: https://books.toscrape.com/catalogue/page-7.html
Scrapeando: https://books.toscrape.com/catalogue/page-8.html
Scrapeando: https://books.toscrape.com/catalogue/page-9.html
Scrapeando: https://books.toscrape.com/catalogue/page-10.html
Scrapeando: https://books.toscrape.com/catalogue/page-11.html
Scrapeando: https://books.toscrape.com/catalogue/page-12.html
Scrapeando: https://books.toscrape.com/catalogue/page-13.html
Scrapeando: https://books.toscrape.com/catalogue/page-14.html
Scrapeando: https://books.toscrape.com/catalogue/page-15.html
Scrapeando: https://books.toscrape.com/catalogue/page-16.html
Scrapeando: https

,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,In stock (19 available),Aaron Ledbetter’s future had been planned out ...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,In stock (18 available),"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,In stock (18 available),A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Poetry,In stock (16 available),"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,19.49,4,Sequential Art,In stock (16 available),"Every summer, Rose goes with her mom and dad t..."
...,...,...,...,...,...,...,...
70,9c96cd1329fbd82d,The Zombie Room,19.69,5,Default,In stock (1 available),An unlikely bond is forged between three men f...
71,b78deb463531d078,The Silent Wife,12.34,5,Fiction,In stock (1 available),A chilling psychological thriller about a marr...
72,4280ac3eab57aa5d,The Girl You Lost,12.29,5,Mystery,In stock (1 available),Eighteen years ago your baby daughter was snat...
73,29fc016c459aeb14,The Edge of Reason (Bridget Jones #2),19.18,4,Womens Fiction,In stock (1 available),Monday 27 January“7:15 a.m. Hurrah! The wilder...
